In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
!{sys.executable} -m pip install sktime[dl]
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt



In [ ]:
def find_dominant_value(array):
    unique, counts = np.unique(array, return_counts=True)
    count_dict = dict(zip(unique, counts))
    return 0 if count_dict.get(0, 0) > count_dict.get(1, 0) else 1

def sawtooth_signal(A, f, t):
    T = 1 / f
    return  A * (t / T - np.floor(t / T))

def square_signal(A, f, t):
    T = 1 / f
    return 0.5*A + 0.5 * A * np.where(np.sin(2 * np.pi * f * t) > 0, 1, -1)

def random_signal(A, f, t, window_size):
    signal = np.zeros(len(t))
    labels = np.zeros(len(t))
    num_windows = int(len(t) // window_size)
    for i in range(num_windows):
        start = i * window_size
        end = (i + 1) * window_size
        if np.random.random() > 0.5:
            signal[start:end] = square_signal(A, f, t[start:end])
            labels[start:end] = 0
        else:
            signal[start:end] = sawtooth_signal(A, f, t[start:end])
            labels[start:end] = 1
    return signal,labels

In [ ]:
t = np.linspace(0, 400*np.pi, 40000)
s,l = random_signal(2, 0.25, t, 200)

fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
axs[0].plot(t[0:1000], s[0:1000])
axs[0].set_title("Signal")
axs[1].plot(t[0:1000], l[0:1000])
axs[1].set_title("Labels")
plt.tight_layout()
plt.show()

In [ ]:
winlen = 200
step = 200
splitter = SlidingWindowSplitter(window_length=winlen, step_length=step)

evlen = len(t)
#logger.info(f"event {evlen}")
S = splitter.split(np.arange(evlen))
Ns = splitter.get_n_splits(np.arange(evlen))
L = list(S)
X = np.zeros([Ns, 1, winlen])
Y = np.zeros([Ns])
for j in range(Ns):
    X[j,0,:] = s[L[j][0]]
    Y[j] = find_dominant_value(l[L[j][0]])

In [ ]:
train_X =  X[0:150,0,:]
test_X =  X[150:199,0,:]
train_y = Y[0:150]
test_y = Y[150:199]
rocket = RocketClassifier(num_kernels=5)
rocket.fit(train_X, train_y)
y_pred = rocket.predict(test_X)

print(accuracy_score(test_y, y_pred))
